# Pendulum Period Analysis
1. Set constants below
2. Run all cells to compute period using different degree Taylor series expansions & ASEBT error bound

In [14]:
# Imports

import numpy as np
from scipy.special import factorial2
import scipy.integrate as integrate
from math import pi
from tabulate import tabulate

In [15]:

# Inputs
L = 1.0                                 # Pendulum length in meters
theta_max_deg = 30                      # Theta max in degrees

# do not modify
theta_max = np.deg2rad(theta_max_deg)   # Convert theta max from degrees to radians


In [16]:
# CONSTANTS
g = 9.80665  # in m/s^2, standard value defined by CIPM
k = np.sin(theta_max / 2)

# 1 * 3 * 5 * 7 * ... * (2n-1) for odd n
# 2 * 4 * 6 * 8 * ... * 2n for even n
# 1 for n = 0 or n = -1
def double_factorial(n: int) -> int:
    if n == 0 or n == -1:
        return 1
    return factorial2(n, exact=True)


# Exact pendulum period using definite integral
def exact_integral() -> float:
    result, _ = integrate.quad(
        # integrand
        lambda theta: 1 / np.sqrt(1 - k**2 * np.sin(theta)**2),
        # bounds of integration
        0,
        np.pi / 2
    )
    return result


def maclaurin_summand(n: int) -> float:
    # (-1)^n * ((2n-1)!! / (2n)!!)^2 * k^(2n) * pi/2
    return (-1)**n * ((double_factorial(2 * n - 1) / double_factorial(2 * n))**2) * k**(2 * n) * (pi / 2)

# Approximate definite integral using Maclaurin series
def mcl_integral_approx(num_terms: int) -> float:
    return sum(maclaurin_summand(n) for n in range(num_terms))


def main(num_terms: int = 5):
    # Compute results
    prefactor = 4 * np.sqrt(L / g)
    exact = prefactor * exact_integral()
    approximations = [prefactor * mcl_integral_approx(num_terms) for num_terms in range(1, num_terms + 1)]
    asebt_errors = [prefactor * maclaurin_summand(num_terms + 1) for num_terms in range(1, num_terms + 1)]

    # Display results in table
    print(f"Results for θ_max = {theta_max_deg}° and L = {L} m:")

    results_table = [
        ["Method", "Period (s)", "Error* (s)"],
        ["Exact", exact, 0],
    ] + [
        [f"Maclaurin Series ({i} terms)", approximations[i-1], asebt_errors[i-1]]
        for i in range(1, num_terms + 1)
    ]

    print(tabulate(results_table, headers="firstrow", floatfmt=".6f", numalign="decimal"))
    print()
    print("* ASEBT error bound")


main()


Results for θ_max = 30° and L = 1.0 m:
Method                        Period (s)    Error* (s)
--------------------------  ------------  ------------
Exact                           2.041338      0.000000
Maclaurin Series (1 terms)      2.006409      0.001266
Maclaurin Series (2 terms)      1.972808     -0.000059
Maclaurin Series (3 terms)      1.974074      0.000003
Maclaurin Series (4 terms)      1.974016     -0.000000
Maclaurin Series (5 terms)      1.974019      0.000000

* ASEBT error bound
